<a href="https://colab.research.google.com/github/jajuangraham/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# Import dependencies
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [2]:

# Load the data
url = "https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv"
application_df = pd.read_csv(url)

# Drop irrelevant columns
application_df = application_df.drop(columns=['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS'])

# Group rare APPLICATION_TYPE values
app_type_counts = application_df["APPLICATION_TYPE"].value_counts()
apps_to_replace = list(app_type_counts[app_type_counts < 500].index)
application_df["APPLICATION_TYPE"] = application_df["APPLICATION_TYPE"].replace(apps_to_replace, "Other")

# Group rare CLASSIFICATION values
class_counts = application_df["CLASSIFICATION"].value_counts()
classes_to_replace = list(class_counts[class_counts < 1000].index)
application_df["CLASSIFICATION"] = application_df["CLASSIFICATION"].replace(classes_to_replace, "Other")

# One-hot encode categorical features
application_df = pd.get_dummies(application_df)


In [3]:

# Define target and features
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"], axis=1).values

# Split and scale data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [4]:

# Build an optimized model
nn_opt = tf.keras.models.Sequential()

# Input + first hidden layer
nn_opt.add(tf.keras.layers.Dense(units=100, activation='relu', input_shape=(X_train.shape[1],)))

# Second hidden layer
nn_opt.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Dropout layer to reduce overfitting
nn_opt.add(tf.keras.layers.Dropout(0.2))

# Output layer
nn_opt.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile model
nn_opt.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:

# Train the model
fit_model = nn_opt.fit(X_train_scaled, y_train, epochs=75)


Epoch 1/75
751/751 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7013 - loss: 0.5910
Epoch 2/75
751/751 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7282 - loss: 0.5577
Epoch 3/75
751/751 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7286 - loss: 0.5546
Epoch 4/75
751/751 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7290 - loss: 0.5565
Epoch 5/75
751/751 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7286 - loss: 0.5509
Epoch 6/75
751/751 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7320 - loss: 0.5505
Epoch 7/75
751/751 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7294 - loss: 0.5554
Epoch 8/75
751/751 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7372 - loss: 0.5476
Epoch 9/75
751/751 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7354 - loss: 0.5463
Epoch 10/75
751/751 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7384 - loss: 0.5444
Epoch 11/75
751/751 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7326 - loss: 0.5489
Epoch 12/75
751/751 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step

In [6]:

# Evaluate the model
model_loss, model_accuracy = nn_opt.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


322/322 - 1s - 2ms/step - accuracy: 0.7288 - loss: 0.5596
Loss: 0.5595555305480957, Accuracy: 0.7287657856941223


In [7]:

# Save the model
nn_opt.save("AlphabetSoupCharity_Optimization.h5")
